In [4]:
import pprint
from pathlib import Path
import fitz

In [46]:
path_pdf = Path("../../example_pdfs/linking.pdf")
path_pdf.is_file()

True

In [40]:
with fitz.open(path_pdf) as doc:
    page = doc[0]
    zoom_x = 1.5
    zoom_y = 1.5
    mat = fitz.Matrix(zoom_x, zoom_y)
    pix = page.get_pixmap(matrix=mat)
    
    # dpi = 300
    # pix = page.get_pixmap(dpi=dpi)
pix

Pixmap(DeviceRGB, IRect(0, 0, 918, 1188), 0)

In [45]:
path_png = Path(f'{path_pdf.stem}_page_0.png')
#path_png = path_pdf.parent/f'{path_pdf.stem}_page_0.png'
pix.save(path_png)

In [34]:
def print_non_under(obj):
    for s in dir(obj):
        if not s.startswith("_"):
            print(s)

In [41]:
print_non_under(pix)

alpha
clearWith
clear_with
color_count
color_topusage
colorspace
copy
copyPixmap
digest
gammaWith
gamma_with
getImageData
getPNGData
getPNGdata
h
height
invertIRect
invert_irect
irect
is_monochrome
is_unicolor
n
pdfocr_save
pdfocr_tobytes
pil_save
pil_tobytes
pillowData
pillowWrite
pixel
samples
samples_mv
samples_ptr
save
setAlpha
setOrigin
setPixel
setRect
setResolution
set_alpha
set_dpi
set_origin
set_pixel
set_rect
shrink
size
stride
this
thisown
tintWith
tint_with
tobytes
w
warp
width
writeImage
writePNG
x
xres
y
yres


In [35]:
pix.width, pix.height

(918, 1188)

## PDF2Image

## Metadata

Real PDFs contain the exact characters appearing in the document as metadata.
Thus, it is wasteful if we do OCR on such documents w/o using these metadata.

- [Convert PDF(s) into Image(s)](https://pymupdf.readthedocs.io/en/latest/recipes-images.html#how-to-make-images-from-document-pages)

In [15]:
with fitz.open("linking.pdf") as doc:
    page = doc[0]
    print(f'{page.mediabox = }')
    print(f'{page.mediabox_size = }')
    #print(f'{dir(page.mediabox_size) = }')
    print(f'{page.mediabox_size.x = }')
    print(f'{page.mediabox_size.y = }')

    blocks = page.get_text("words", sort=True)

print("blocks =")
pprint.pprint(blocks)

page.mediabox = Rect(0.0, 0.0, 612.0, 792.0)
page.mediabox_size = Point(612.0, 792.0)
page.mediabox_size.x = 612.0
page.mediabox_size.y = 792.0
blocks =
[(133.76800537109375,
  124.00536346435547,
  141.83056640625,
  138.36590576171875,
  '1',
  0,
  0,
  0),
 (157.98439025878906,
  124.00536346435547,
  213.96327209472656,
  138.36590576171875,
  'Current',
  0,
  1,
  0),
 (219.35743713378906,
  124.00536346435547,
  246.4717559814453,
  138.36590576171875,
  'way',
  0,
  1,
  1),
 (251.8659210205078,
  124.00536346435547,
  264.86358642578125,
  138.36590576171875,
  'of',
  0,
  1,
  2),
 (270.25775146484375,
  124.00536346435547,
  308.8203125,
  138.36590576171875,
  'doing',
  0,
  1,
  3),
 (137.08900451660156,
  151.73703002929688,
  148.70465087890625,
  161.6996307373047,
  '(0,',
  1,
  0,
  0),
 (150.37200927734375,
  151.73703002929688,
  159.2187957763672,
  161.6996307373047,
  '0)',
  1,
  0,
  1),
 (178.5760040283203,
  151.73703002929688,
  195.1726531982422,
  161